In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# %matplotlib widget


In [192]:
# Standard Library Imports
import os
import csv
from datetime import datetime, timedelta, timezone
from collections import defaultdict
from collections import OrderedDict
from itertools import product
from itertools import groupby
import itertools
import pickle
import glob
import chardet

# Third-Party Library Imports
import numpy as np
import matplotlib.pyplot as plt
# Set a professional style for the plot
plt.style.use('_mpl-gallery')
import matplotlib.dates as mdaates
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset, plot_trigger
from obspy import Trace
from obspy.imaging.spectrogram import spectrogram
import pandas as pd
from geopy.distance import geodesic
from tqdm.auto import tqdm
from icecream import ic
import seaborn as sns
from scipy.stats import norm, kstest, skew, kurtosis
import warnings
warnings.filterwarnings("ignore")

# Local Imports
from energy.utils_general import *
from energy.utils_energy import *
from energy.preprocessing import *
from energy.metrics import *

In [156]:
# Get the path to all the folders from the folder BD paper that is in the current directory
folders = glob.glob('BD paper/*')
# Get all the folders in the BD paper folder that start with waveform 
folders_signals = [folder for folder in folders if folder.startswith('BD paper/waveform')]
folders_signals.sort()

In [160]:
def z_channel_dict(folders_signals):
    result_dict = {}

    for folder in folders_signals:
        waveform_type = os.path.basename(folder)
        for event_path in os.scandir(folder):
            if event_path.is_dir():
                events_name = event_path.name
                for network_folder in os.scandir(event_path.path):
                    if network_folder.is_dir():
                        network_name = network_folder.name
                        stations_z_channel_z = [station_z.path for station_z in os.scandir(network_folder.path) if 'BHZ' in station_z.name]
                        traces = [read(station_z_path)[0] for station_z_path in stations_z_channel_z]
                        result_dict.setdefault(waveform_type, {}).setdefault(events_name, {})[network_name] = traces

    return result_dict

result_dict = z_channel_dict(folders_signals)


In [178]:

def z_channel_dict(folders_signals):
    result_dict = OrderedDict()

    # Define a custom sorting key function that sorts by magnitude and then by date
    def sorting_key(d):
        name_parts = d.name.split('_')
        magnitude = float(name_parts[0])
        date = datetime.strptime('_'.join(name_parts[1:]), '%Y-%m-%d_%H.%M.%S')
        return (-magnitude, date)  # Use negative magnitude for descending order

    for folder in sorted(folders_signals):
        waveform_type = os.path.basename(folder)
        if waveform_type not in result_dict:
            result_dict[waveform_type] = OrderedDict()
        for event_path in sorted(os.scandir(folder), key=sorting_key):
            if event_path.is_dir():
                events_name = event_path.name
                if events_name not in result_dict[waveform_type]:
                    result_dict[waveform_type][events_name] = OrderedDict()
                for network_folder in sorted(os.scandir(event_path.path), key=lambda d: d.name):
                    if network_folder.is_dir():
                        network_name = network_folder.name
                        stations_z_channel_z = [station_z.path for station_z in sorted(os.scandir(network_folder.path), key=lambda d: d.name) if 'BHZ' in station_z.name]
                        traces = [read(station_z_path)[0] for station_z_path in stations_z_channel_z]
                        result_dict[waveform_type][events_name][network_name] = traces

    return result_dict

result_dict1 = z_channel_dict(folders_signals)

In [179]:
result_dict1['waveform 4p1'].keys()

odict_keys(['4.3_2014-01-07_22.45.39', '4.3_2014-02-17_00.47.08', '4.3_2014-03-04_02.49.45', '4.3_2014-03-09_01.49.04', '4.3_2014-03-17_04.59.21', '4.3_2014-03-17_11.46.35', '4.3_2014-03-24_07.13.50', '4.3_2014-03-24_18.20.28', '4.3_2014-03-24_21.08.30', '4.2_2014-03-25_21.58.48', '4.2_2014-04-02_00.10.33', '4.2_2014-04-02_07.42.21', '4.2_2014-04-02_08.13.07', '4.2_2014-04-02_09.21.37', '4.2_2014-04-02_10.18.22', '4.1_2014-04-02_17.05.15', '4.1_2014-04-02_17.06.54', '4.1_2014-04-02_19.57.47', '4.1_2014-04-03_03.28.50', '4.1_2014-04-03_08.44.59', '4.1_2014-04-03_10.00.52', '4.1_2014-04-03_14.01.04', '4.0_2014-04-03_20.24.39', '4.0_2014-04-03_23.13.17', '4.0_2014-04-04_03.19.40', '4.0_2014-04-04_07.42.17', '4.0_2014-04-04_07.57.32', '4.0_2014-04-05_01.36.12', '4.0_2014-04-05_02.05.44', '4.0_2014-04-05_11.03.30', '4.0_2014-04-05_12.15.57'])

## La siguiente función pone a todas las traces de TODOS los eventos en una fila, esto lo tengo a priori, quizás habrá que arreglarlo

In [182]:
def get_all_traces(nested_dict):
    traces = []
    for value in nested_dict.values():
        if isinstance(value, dict):
            traces.extend(get_all_traces(value))
        else:
            traces.extend(value)
    return traces

# Get all ObsPy Trace objects in the dictionary
all_traces = get_all_traces(result_dict1)

In [184]:
folders

['BD paper/waveform 4p1',
 'BD paper/waveform 5',
 'BD paper/lista de datos',
 'BD paper/waveform 4p2',
 'BD paper/waveform 6-9']

In [195]:

file_path = os.path.join('BD paper/lista de datos', 'Eventos 5.csv')

# Detectar el encoding del archivo
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

# Crear un dataframe a partir de tu archivo csv con el encoding detectado
df = pd.read_csv(file_path, delimiter='|', encoding=result['encoding'])

# Seleccionar solo las columnas que necesitas
df = df[["Time", "Latitude", "Longitude", "Magnitude"]]

# Guardar el nuevo dataframe en un archivo Excel
df.to_excel(os.path.join('BD paper/lista de datos','nuevo_archivo.xlsx'), index=False)